In [ ]:
import unittest
from unittest.mock import Mock, patch
from io import BytesIO
import pandas as pd
from jobs.gold_layer import GoldLayerProcessor
from plugins.storage_client import AzureStorageClient
from plugins.spark_manager import SparkManager
from pyspark.sql import SparkSession
from pyspark.sql import Row


class TestGoldLayerProcessor(unittest.TestCase):

    def setUp(self):
        self.mock_storage_client = Mock(spec=AzureStorageClient)
        self.mock_spark_manager = Mock(spec=SparkManager)

        self.mock_spark = Mock(SparkSession)
        self.mock_spark_manager.create_spark_session.return_value = self.mock_spark

        self.mock_schema = Mock()
        self.mock_spark_manager.get_breweries_schema.return_value = self.mock_schema

        self.processor = GoldLayerProcessor(
            storage_client=self.mock_storage_client,
            spark_manager=self.mock_spark_manager
        )

    @patch('pyspark.sql.DataFrame')
    def test_process(self, mock_df):

        test_parquet_data = pd.DataFrame({
            'country': ['US'],
            'state': ['NY'],
            'city': ['New York'],
            'brewery_type': ['micro'],
            'brewery_count': [10]
        })

        self.mock_storage_client.download_blob.return_value = test_parquet_data

        self.mock_spark.createDataFrame.return_value = mock_df
        
        mock_df.groupBy.return_value.agg.return_value.orderBy.return_value = mock_df
        mock_df.toPandas.return_value = test_parquet_data

        self.processor.process()

        self.mock_storage_client.download_blob.assert_called_once_with(
            container='silver-layer', blob_path='breweries_data_processed', file_type='parquet'
        )
        mock_df.groupBy.assert_called_once_with(
            "country", "state", "city", "brewery_type")
        
        mock_df.agg.assert_called_once()

        self.mock_storage_client.upload_blob.assert_called_once()
        self.mock_spark.createDataFrame.assert_called_once()


if __name__ == '__main__':
    unittest.main()